# LES IMPORT NECESSAIRE 🧮

In [1]:
import tensorflow as tf                                                                          # type: ignore
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer                                        # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences                                # type: ignore
from tensorflow.keras.models import Model                                                        # type: ignore
from tensorflow.keras.layers import Input, Embedding, Layer, Dense, Dropout, Bidirectional, LSTM # type: ignore

# CONTENU DU CORPUS

In [2]:
with open('data/poeme.txt', 'r', encoding='utf8') as f:
    corpus = f.readlines()
# Limiter à 20 lignes
corpus = corpus[:]
def affich_lignes(n_lignes, texts):
    for i in range(n_lignes):
        print(i, texts[i])
        
affich_lignes(8, corpus)

print("Nombre de lignes:", len(corpus))
print("Nombre de mots:", sum(len(ligne.split()) for ligne in corpus))
print("Nombre de caractères:", sum(len(ligne) for ligne in corpus))

0 Le reve pousse le cœur à croire.

1 La croyance nourrit la force du courage.

2 Une force grandit quand l’effort devient constant.

3 Un effort sincère transforme l’echec en chance.

4 Le doute recule devant une volonte ferme.

5 La volonte trace un chemin vers demain.

6 Le chemin longs forgent l’âme en silence.

7 Une âme stable traverse les grandes tempetes.

Nombre de lignes: 30
Nombre de mots: 212
Nombre de caractères: 1273


# PRETRAITEMENT ET TOKENIZATION

In [3]:
input_texts = ["bonjour je veux manger du code", "Un code exemple pour le corpus du data"]
input_texts += corpus[:30] # ajout de quelques lignes du corpus
def preprocess(texts):
    return ["<START> " + t.lower().strip() + " <END>" for t in texts]
input_texts = preprocess(input_texts)
input_tokenizer = Tokenizer(filters='', lower=True, oov_token='<OOV>')
input_tokenizer.fit_on_texts(input_texts)

input_seqs = input_tokenizer.texts_to_sequences(input_texts)
input_tensor = pad_sequences(input_seqs, padding='post')

print(input_texts)
print(input_tokenizer.word_index)
print('input_seqs:', input_seqs)
print('\ntenseur final:\n', input_tensor)

['<START> bonjour je veux manger du code <END>', '<START> un code exemple pour le corpus du data <END>', '<START> le reve pousse le cœur à croire. <END>', '<START> la croyance nourrit la force du courage. <END>', '<START> une force grandit quand l’effort devient constant. <END>', '<START> un effort sincère transforme l’echec en chance. <END>', '<START> le doute recule devant une volonte ferme. <END>', '<START> la volonte trace un chemin vers demain. <END>', '<START> le chemin longs forgent l’âme en silence. <END>', '<START> une âme stable traverse les grandes tempetes. <END>', '<START> un pas difficile mène vers la lumière. <END>', '<START> le temps revèle le fruit du travail. <END>', '<START> la patience soutient les actions de verite. <END>', '<START> les erreurs enseignent des leçons durables et fortes. <END>', '<START> une victoire attend derrière chaque pas vrai. <END>', '<START> un esprit libre cree sa propre reussite. <END>', '<START> la reussite s’offre à ceux qui persevèrent. 

# VECTORISATION ET EMBEDDINGS

In [4]:
vocab_size = len(input_tokenizer.word_index) + 1
embed_dim = min(600, int(vocab_size ** 0.25 * 10))

print('vocab_size', vocab_size, 'embed_dim', embed_dim)
model = tf.keras.Sequential([Embedding(input_dim=vocab_size, output_dim=embed_dim)])

output_tensor = model(input_tensor)
print('output_tensor shape:', output_tensor.shape)
print('un seul Emedding :',output_tensor[0][0])

vocab_size 141 embed_dim 34
output_tensor shape: (32, 10, 34)
un seul Emedding : tf.Tensor(
[-3.7881304e-02  3.0145023e-02  1.1187542e-02  4.9914729e-02
  1.9710992e-02 -3.8899601e-02  2.8979469e-02  3.6528375e-02
  4.7872964e-02 -4.4871163e-02 -2.6559507e-02  1.1027396e-02
 -1.1290003e-02  2.7563822e-02 -1.9980479e-02  2.3805704e-02
 -4.7558080e-02 -3.5211362e-02  5.4070465e-03  7.9777837e-03
  1.2626205e-02 -8.5674748e-03 -4.2720355e-02  3.8236976e-03
  2.3961067e-05  3.7299756e-02  3.3295956e-02  1.5860323e-02
 -4.9466502e-02 -1.5279949e-02  4.1144315e-02 -4.1215383e-02
 -7.8428276e-03  3.5392750e-02], shape=(34,), dtype=float32)


# GENERATION DONNEES D'ENTRAINEMENT (SELF-SUPERVISED)

In [5]:
def list_to_xy(input_seqs):
    dataxy = []
    for seq in input_seqs:
        for i in range(len(seq)):
            data_ligne = seq[:i+1]
            dataxy.append(data_ligne)
    return dataxy

def print_inline(list_data):
    for data in list_data:
        print(data)
    print('etc...')

print(input_seqs)
dataxy = list_to_xy(input_seqs)
print_inline(dataxy[:15])

[[2, 34, 35, 36, 37, 9, 15, 3], [2, 8, 15, 38, 39, 4, 40, 9, 41, 3], [2, 4, 42, 43, 4, 16, 17, 44, 3], [2, 5, 45, 46, 5, 12, 9, 47, 3], [2, 7, 12, 48, 49, 50, 18, 51, 3], [2, 8, 52, 53, 54, 55, 13, 19, 3], [2, 4, 56, 57, 58, 7, 14, 59, 3], [2, 5, 14, 60, 8, 10, 11, 61, 3], [2, 4, 10, 62, 20, 63, 13, 64, 3], [2, 7, 65, 66, 67, 6, 68, 69, 3], [2, 8, 21, 70, 22, 11, 5, 71, 3], [2, 4, 23, 72, 4, 73, 9, 24, 3], [2, 5, 25, 74, 6, 75, 76, 77, 3], [2, 6, 26, 78, 79, 80, 81, 82, 27, 3], [2, 7, 28, 83, 84, 85, 21, 86, 3], [2, 8, 87, 88, 89, 90, 91, 29, 3], [2, 5, 30, 92, 17, 93, 31, 94, 3], [2, 4, 95, 96, 4, 10, 9, 97, 3], [2, 5, 25, 98, 5, 14, 9, 99, 3], [2, 6, 32, 20, 6, 33, 13, 100, 3], [2, 7, 101, 102, 7, 103, 104, 105, 3], [2, 8, 106, 107, 108, 7, 109, 19, 3], [2, 4, 110, 111, 5, 112, 9, 113, 3], [2, 5, 30, 114, 5, 115, 9, 24, 3], [2, 6, 26, 116, 6, 117, 6, 118, 27, 3], [2, 7, 28, 119, 120, 7, 121, 122, 3], [2, 8, 123, 18, 8, 124, 11, 125, 3], [2, 4, 23, 126, 4, 16, 31, 127, 3], [2, 5, 12, 

In [6]:
# application du padding
max_seq_len = max(len(seq) for seq in input_seqs)
dataxy = pad_sequences(dataxy, padding='pre', maxlen=max_seq_len)
print_inline(dataxy[:15])

[0 0 0 0 0 0 0 0 0 2]
[ 0  0  0  0  0  0  0  0  2 34]
[ 0  0  0  0  0  0  0  2 34 35]
[ 0  0  0  0  0  0  2 34 35 36]
[ 0  0  0  0  0  2 34 35 36 37]
[ 0  0  0  0  2 34 35 36 37  9]
[ 0  0  0  2 34 35 36 37  9 15]
[ 0  0  2 34 35 36 37  9 15  3]
[0 0 0 0 0 0 0 0 0 2]
[0 0 0 0 0 0 0 0 2 8]
[ 0  0  0  0  0  0  0  2  8 15]
[ 0  0  0  0  0  0  2  8 15 38]
[ 0  0  0  0  0  2  8 15 38 39]
[ 0  0  0  0  2  8 15 38 39  4]
[ 0  0  0  2  8 15 38 39  4 40]
etc...


# IMPLEMENTATION PAR UN LSTM 📌

In [7]:

import tensorflow as tf
# definir x_train et y_train
x_train, y_train = dataxy[:, :-1], dataxy[:, -1]

# Reshape pour correspondre à la forme attendue
y_train = y_train.reshape(-1, 1)
print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)

# convertir en categorical position parmi les classes 
y_train = tf.keras.utils.to_categorical(y_train, num_classes=vocab_size)
print('y_train shape:', y_train.shape)
print('premier exemple de y_train:', y_train[0])
print('embed_dim', embed_dim, 'vocab_size', vocab_size)

# Implémentation du LSTM
model = tf.keras.models.Sequential([
    Input(shape=(None,)),
    Embedding(input_dim=vocab_size, output_dim=embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

x_train shape: (290, 9) y_train shape: (290, 1)
y_train shape: (290, 141)
premier exemple de y_train: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
embed_dim 34 vocab_size 141


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, None, 34)       │         4,794 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        50,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 141)            │        18,189 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,927 (288.78 KB)

 Trainable params: 73,927 (288.78 KB)

 Non-trainable params: 0 (0.00 B)

# ENTRAINEMENT DU MODEL

In [8]:
import torch
print(torch.__version__)

2.7.0+cpu


In [9]:
model.fit(x_train, y_train, epochs=600, batch_size=1024, verbose=1)

Epoch 1/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0103 - loss: 4.9536
Epoch 2/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0241 - loss: 4.8364
Epoch 3/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.0862 - loss: 4.7237
Epoch 4/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1483 - loss: 4.5922
Epoch 5/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.1448 - loss: 4.4918
Epoch 6/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1897 - loss: 4.4225
Epoch 7/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.2000 - loss: 4.3873
Epoch 8/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2172 - loss: 4.2752
Epoch 9/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.2207 - loss: 4.2633
Epoch 10/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2172 - loss: 4.1946
Epoch 11/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2207 - loss: 4.1872
Epoch 12/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.2241 - loss

# INFERENCE LSTM

In [15]:
prompt = input("🟡🔵 Entrez un prompt: ")
max_length = int(input("🟢🔴 Entrez la longueur de generation :"))
def process_prompt(prompt):
    prompt = preprocess([prompt])
    #print(prompt)
    prompt_seq = input_tokenizer.texts_to_sequences(prompt)
    #print('prompt_seq', prompt_seq)
    prompt_x = pad_sequences(prompt_seq, padding='pre', maxlen=max_seq_len)
    #print('prompt_seq shape: ', prompt_x.shape)
    #print(prompt_x)
    return prompt_x

def mot_suivant(prompt):
    prompt_x = process_prompt(prompt)
    pred = np.argmax(model.predict(prompt_x))
    # print(pred)
    # print(input_tokenizer.word_index)
    mot_suivant = input_tokenizer.index_word[pred]
    return mot_suivant

#print(prompt)
#next_token = mot_suivant(prompt)
#print(next_token)

def generate_text(prompt, max_length):
    for i in range(max_length):
        prompt = prompt + ' '
        next_token = mot_suivant(prompt)
        prompt += next_token
        if prompt.split()[-1] == '<end>':
            break
    return prompt
print(generate_text(prompt, max_length))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
le bonheur qui pousse le cœur à croire.


# IMPLEMENTATION DU TRANSFORMER 🎯

Positionnal Encoding

In [ ]:
# determiner le max_seq_len
input_seqs = input_tokenizer.texts_to_sequences(input_texts)
max_seq_len = max(len(seq) for seq in input_seqs)
positions = tf.range(max_seq_len)
print('max_seq_len:', max_seq_len)

pos_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
positions = tf.range(start=0, limit=max_seq_len, delta=1)
pos_vectors = pos_embedding(positions)  # (seq_len, embed_dim)
print('Positionnal Encoding shape:', pos_vectors.shape)
print('Positionnal Encoding:', pos_vectors[0])

max_seq_len: 10
Positionnal Encoding shape: (10, 20)
Positionnal Encoding: tf.Tensor(
[-0.03520043 -0.04310166 -0.03162181 -0.00244892  0.00445268 -0.0011786
 -0.04042745  0.04138353 -0.00614424  0.03153454  0.032522    0.03013242
 -0.04533926  0.02421163  0.00258048 -0.02930994  0.00017513 -0.03202363
 -0.01583755  0.02624207], shape=(20,), dtype=float32)


In [ ]:
# classes pour le transformer
def get_transformer_encoder(embed_dim, num_heads, ff_dim):
	# couche d'entrée
	inputs = Input(shape=(None, embed_dim))
	# MultiHeadAttention
	attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
	attention_output = tf.keras.layers.Dropout(0.1)(attention_output)
	attention_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention_output)
	# Feed Forward
	ffn = tf.keras.Sequential([
		Dense(ff_dim, activation="relu"),
		Dense(embed_dim),
	])
	ffn_output = ffn(attention_output)
	ffn_output = tf.keras.layers.Dropout(0.1)(ffn_output)
	encoder_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)
	return tf.keras.Model(inputs=inputs, outputs=encoder_output)